In [ ]:
from choquet_integral import *
import numpy as np
import itertools
import math
import inspect
import matplotlib.pyplot as plt
import csv
import sklearn.metrics as metrics
from sklearn import datasets
from sklearn.preprocessing import scale
import pandas
from pyclustertend import vat, ivat
import matplotlib.pyplot as plt

In [ ]:
# our integral
chi = ChoquetIntegral()

# load the training file 
# 02.txt: https://github.com/aminb99/remote-sensing-nn-datasets/blob/master/AID_4net_dataset/NNFold_1/Fold_1/test/02.txt
# _02.txt: https://github.com/aminb99/remote-sensing-nn-datasets/blob/master/AID_dataset/NNFold_1/Fold_1/test/02.txt
df = pandas.read_csv('02.txt')
df

# how many nets? 
n = 4 # 7

In [ ]:
# suck out labels and data
training_labels = np.asarray( df.values )[:,n]
training_data = np.asarray( df.values )[:,0:n]

In [ ]:
# for class ?, make this a binary learning problem
locs = np.where(training_labels == 1)
print(locs)
labels = np.zeros(training_labels.shape)
labels[locs] = 1

In [ ]:
# train the integral
chi.train_chi(training_data.transpose(), labels)
print(chi.fm)

In [ ]:
#distance matrix code from Matt
initial = []
for i in range(n):
    initial.append(i+1)
    
a = itertools.permutations(initial,n)
b = np.ndarray((math.factorial(n),n),dtype=int)
c = np.ndarray((math.factorial(n),n),dtype=float)
for i,val in enumerate(a):
    b[i] = np.asarray(val)
    
    c_build = np.ndarray(n)
    for j in range(len(b[i])):
        if j == 0:
            c_build[j] = chi.fm[str(b[i][0:1])]
        else:
            c_build[j] = (chi.fm[str(np.sort(b[i][0:j+1]))]) - (chi.fm[str(np.sort(b[i][0:j]))])

    c[i] = c_build

In [ ]:
C = c
b = vat(C, return_ODM=True)
a = ivat(C, return_D_prim=True)

In [ ]:
# I want to plot it

#vat
plt.figure()
color_map = plt.imshow(a)
color_map.set_cmap("gray")
#plt.colorbar()

#ivat - dang, an't as straight forward due to transform (I will have to think about!)
plt.figure()
color_map = plt.imshow(a,vmin=0,vmax=1)
color_map.set_cmap("gray")

In [ ]:
# find the unique sorts
sortorder = np.argsort( (-1) * training_data )
print(sortorder)
unique_rows = np.unique(sortorder, axis=0)
print(unique_rows)
Nunique = unique_rows.shape[0]
print(Nunique)

In [ ]:
#distance matrix on just unique sorts
d = np.ndarray((Nunique,n),dtype=float)
for i in range(Nunique):
    b = np.asarray(unique_rows[i]) + 1
    c_build = np.zeros(n)
    for j in range(n):
        if j == 0:
            c_build[j] = chi.fm[str(b[0:1])]
        else:
            c_build[j] = (chi.fm[str(np.sort(b[0:j+1]))]) - (chi.fm[str(np.sort(b[0:j]))])
    d[i] = c_build

In [ ]:
# what are these operators? plot them out
np.set_printoptions(formatter={'float': '{: 0.9f}'.format})
for i in range(Nunique):
    print(d[i])

In [ ]:
#dissimilarity matrix, i want to see it unsorted
pwd = metrics.pairwise_distances(d,metric='euclidean')
color_map = plt.imshow(pwd)
color_map.set_cmap("gray")
plt.colorbar()

In [ ]:
# vat and ivat on just the seen walks
C = d
b = vat(C, return_ODM=True)
a = ivat(C, return_D_prim=True)

In [ ]:
print(d)
# naive extraction
ListOfOps = np.asarray([0])
Thresh = 0.1
for i in range(1,Nunique):
    dist = np.zeros( ListOfOps.shape[0] )
    for j in range(ListOfOps.shape[0]):
        dist[j] = np.sqrt(np.sum(np.power(d[i]-d[ListOfOps[j]],2.0)))
    # new...?
    # look at the thing that its closest to, is it greater than some threshold?
    if( np.amin(dist) > Thresh ):
        ListOfOps = np.append(ListOfOps,i)
# here are our operators ...
print(ListOfOps)
print(d[ListOfOps])